In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
import torch
from PIL import Image
from transformers import BlipProcessor, BlipForQuestionAnswering

# Cek perangkat
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load processor dan model
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base").to(device)

# Fungsi inferensi
def answer_question(image: Image.Image, question: str) -> str:
    image = image.convert('RGB')  # Pastikan gambar RGB
    inputs = processor(image, question, return_tensors="pt").to(device)

    with torch.no_grad():
        output_ids = model.generate(**inputs)

    answer = processor.tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return answer

# Layout Gradio dengan desain modern
with gr.Blocks(theme=gr.themes.Soft(), title="Visual Question Answering - BLIP") as demo:
    gr.Markdown(
        """
        <div style="text-align: center; padding: 10px;">
            <h1 style="font-size: 2.5rem;">🧠📷 Visual Question Answering</h1>
            <p style="font-size: 1.2rem; color: #555;">
                Unggah gambar dan ajukan pertanyaan. Model <b>BLIP</b> akan memberikan jawaban berdasarkan isi gambar.
            </p>
        </div>
        """,
        elem_id="header"
    )

    with gr.Row(equal_height=True):
        with gr.Column(scale=1, min_width=400):
            image_input = gr.Image(type="pil", label="Upload Image", show_label=True, height=300)
            question_input = gr.Textbox(label="Your Question", placeholder="e.g., What is in the picture?", lines=1)
            submit_btn = gr.Button("🔍 Get Answer", variant="primary", size="lg")
        with gr.Column(scale=1, min_width=400):
            answer_output = gr.Textbox(label="Answer", placeholder="Model's answer will appear here", lines=3)

    submit_btn.click(fn=answer_question, inputs=[image_input, question_input], outputs=answer_output)

# Jalankan aplikasi
demo.launch()
